In [1]:
import plasmapy

Note that plasmapy can take a very long time to import, about a minute.

In [78]:
import hii_utils
import plasmapy.formulary as pf
import astropy.units as u
import astropy.constants as const
import numpy as np

## Calculate various parameters for a typical H II region

### Fiducial parameters

As with my other notes, we will normalise to an electron density of 100 pcc and temperature of 10,000 K. We set the size scale to 10 pc (later, we can try and relate this to the ionization parameter)

In [3]:
L = 10.0 * u.pc
n = 100.0 * u.cm ** -3
T = 1.0e4 * u.K
species = ("p+", "p+")

### Magnetic field

Set the magnetic field to try and give a plasma beta of unity. Note that the `n` parameter in the `beta()` function is the total particle density, which is $2 n_e$ if fully ionized pure hydrogen.

In [4]:
B = 83.3* u.microgauss
pf.beta(T, 2*n, B)

<Quantity 1.00014539>

That is close enough to unity. But note that it is slightly different from the 78.2 microG that I derived in my notes, which is probably due to the correction for Helium, which I am ignoring here.

### Collisional quantities

In [5]:
pf.Knudsen_number(L, T, n, species)

<Quantity 1.79932417e-10>

#### Mean free path, thermal speed, and collisional frequencies

Mean free path comes out the same for electrons and protons

In [31]:
ion_mfp = pf.mean_free_path(T, n, species)
ion_mfp.cgs

<Quantity 5.55213426e+09 cm>

But it is slightly different for Helium ions

In [7]:
pf.mean_free_path(T, n, ("He++", "p+")).cgs

<Quantity 1.43652977e+09 cm>

And it is a *lot* bigger for high-velocity protons (for example, from stellar wind)

In [8]:
pf.mean_free_path(T, n, species, V=1000 * u.km/u.s).cgs

<Quantity 3.66291304e+16 cm>

Calculate the thermal speed. This is the 3d rms speed, but I am not sure if this is exactly what we should be using.

In [34]:
v_rms = pf.thermal_speed(T, "p", method="rms", ndim=3)
v_rms.to(u.km/u.s)

<Quantity 15.73632717 km / s>

OK, I have investigated more and it turns out it uses the most probable speed from 3d Maxwellian by default, which is slightly lower

In [84]:
v_therm_ion = pf.thermal_speed(T, "p", ndim=3)
v_therm_ion.to(u.km/u.s)

<Quantity 12.84865733 km / s>

Now get the collisional time. Supposedly, we should use the MaxwellianCollisionFrequencies class, but that is a bit of a pain since it requires sending the Coulomb logarithm. But first we will try the old deprecated version:

In [60]:
nu_i_old = pf.fundamental_ion_collision_freq(T, n, "p")
nu_i_old

<Quantity 0.00011894 1 / s>

And now we calculate the Coulomb logarithm and use the new way of doing it

In [61]:
pf.Coulomb_logarithm(T, n, species)

np.float64(20.532051566138307)

In [62]:
freqs = pf.MaxwellianCollisionFrequencies(
    "p", "p", T_a=T, n_a=n, T_b=T, n_b=n,
    Coulomb_log=pf.Coulomb_logarithm(T, n, species) * u.dimensionless_unscaled,
)

In [63]:
nu_i = freqs.Maxwellian_avg_ii_collision_freq
nu_i

<Quantity 4.35212736e-05 Hz>

There is also a simpler tyoe of collision frequency that comes out differently

In [68]:
nu_Lorentz = freqs.Lorentz_collision_frequency
nu_Lorentz

<Quantity 8.18187412e-05 Hz>

In [74]:
nu_Lorentz_old = pf.collision_frequency(T, n, species)
nu_Lorentz_old

<Quantity 0.00032727 Hz>

Or converted into a time:

In [99]:
tau_i = 1/nu_Lorentz_old
tau_i.cgs

<Quantity 3055.53466178 s>

In [85]:
(ion_mfp * nu_Lorentz_old).to(u.km/u.s)

<Quantity 18.17074545 km / s>

OK, so that varies a lot, depending on whether we use the Lorentz or Maxwellian and whether we use the class (new) or the function (old). Looking at the source code for mean_free_path, it uses the Lorentz function version, so we will do the same for consistency.

In [86]:
np.sqrt(2) * v_therm_ion.to(u.km/u.s)

<Quantity 18.17074545 km / s>

So that is consistent, yay!  The reason for the sqrt(2) factor is that it is the relative speed between two protons. 

#### Viscosity

By default, the transport coefficients are calculated in the Braginskii model

In [9]:
pf.ion_viscosity(T, n, T, n, "p")

<Quantity [1.07673276e-07, 1.07345943e-07, 1.07345943e-07, 0.00000000e+00,
           0.00000000e+00] Pa s>

But there is the option to use the more accurate Ji and Held (2013) model, although in our case this makes little difference.

But we can also a more recent theory from Ji & Held (2013), which changes the numbers very slightly. The main difference is that it better accounts for the electron-ion collisions, and apparently it has more effect in the magnetic case, which we will  see below. 

In [10]:
pf.ion_viscosity(T, n, T, n, "p", model="Ji-Held")

<Quantity [1.08256591e-07, 1.08256591e-07, 1.08256591e-07, 0.00000000e+00,
           0.00000000e+00] Pa s>

Note that viscosity is a tensor, which may become important once we introduce a magnetic field.

#### Anisotropy with magnetic field

In [30]:
hall = pf.Hall_parameter(n, T, B, "p", "p")
hall

<Quantity 6708.50398785>

The Hall parameter is the product of Larmor frequency and collision time: $x = \omega_L \, \tau = (V / r_L)\, (\lambda / V) = \lambda / r_L$. so it is the ratio of the mean free path to the Larmor radius. This is very similar to the magnetic Prandtl number, which I calculated in the notes. It is linearly proportional to the B field:

In [12]:
pf.Hall_parameter(n, T, 0.1 * B, "p", "p")

<Quantity 670.85039879>

In [13]:
pf.ion_viscosity(T, n, T, n, "p", B=B, model="Ji-Held")

<Quantity [1.08256591e-07, 1.66132734e-23, 6.64530909e-23, 1.24610698e-15,
           2.49221389e-15] Pa s>

In [15]:
viscosity = pf.ion_viscosity(T, n, T, n, "p", B=B)
viscosity

<Quantity [1.07673276e-07, 1.66132734e-23, 6.64530913e-23, 1.24610698e-15,
           2.49221389e-15] Pa s>

These are now the tensor viscosity results with a more realistic magnetic field

In [17]:
viscosity[0] / viscosity[3]

<Quantity 86407730.2416565>

Why is there such a large difference between these components? I need to better understand the relation with the parallel, perpendicular, and cross components.

In [43]:
pf.electron_viscosity(T, n, T, n, "p", B=B)

<Quantity [1.35666080e-09, 6.37402018e-31, 2.54960807e-30, 2.39890714e-20,
           4.79781428e-20] Pa s>

So the electron viscosity (for the *ions*) is 100  times lower than the ion viscosity, so we can ignore it. 

#### Kinematic viscosity

The kinematic viscosity is equal to the dynamic viscosity divided by the mass density. 

In [29]:
kinematic_viscosity = viscosity / (n * const.m_p)
kinematic_viscosity.cgs

<Quantity [6.43739474e+15, 9.93247380e-01, 3.97298938e+00, 7.45002180e+07,
           1.49000432e+08] cm2 / s>

This should be equal to the mean-free path times the ion thermal speed. 

Let's make the comparison:

In [87]:
(ion_mfp * v_therm_ion).cgs

<Quantity 7.13374705e+15 cm2 / s>

Hurray, that is very close to what I was expecting for the parallel viscosity. Some texts have a factor of 1/3 in this equation, but if I used that, then we would have a bigger discrepancy.

### Reynolds number

This is what we finally want to calculate.

Consider pure hydrogen and fully ionized to calculate the isothermal sound speed. 

In [96]:
sound_speed = np.sqrt(2 * const.k_B * T / const.m_p)
sound_speed.to(u.km / u.s)

<Quantity 12.84865733 km / s>

This is actually numerically equal to the ion thermal speed. 

I use the parallel component of the dynamic viscosity. 

In [97]:
Re = pf.Reynolds_number(
    rho = const.m_p * n,
    U = v_therm_ion,
    L = L,
    mu = viscosity[0],
)
Re

<Quantity 6.15882907e+09>

The integral scale Re for H II regions will be a bit smaller since the injection scale will be smaller than the Stromgren radius. But it will still be of order 1e9. 